In [119]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline
import scipy.stats as st
import pprint
# Import API key
# with open('/Users/tmlun/Downloads/api_keys.py', 'r') as g_key:
#     mykey=g_key.read()
#     mykey=mykey.split('=')
#     mykey=mykey[1].strip()   
# g_key = mykey.strip('"')
#from api_keys import g_key
g_key = ""
gmaps.configure(api_key=g_key)

#Data File path (CSV)
data = "./output_data/latlng.csv"
data2 = "./output_data/month_grp.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [120]:
# Study data files
RMS_path = "./RMS.csv"
# Read the data
Crime_df = pd.read_csv(RMS_path)
print(Crime_df)


       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
2         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219220  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219221  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219223  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [121]:
#Clean the Zipcode Data
Crime_df = Crime_df.dropna(subset=['ZIP'])
Crime_df = Crime_df.loc[(Crime_df["OFFENSE-DESC"]!="Other Crimes")]
print(Crime_df)

       ARREST_DATE        DATE  YEAR     MONTH  WEEK SEX RACE ETHNIC  FBI  \
0         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      N  NaN   
1         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
3         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
4         1/1/2016    1/1/2016  2016   JANUARY     1   F    W      H  NaN   
5         1/1/2016    1/1/2016  2016   JANUARY     1   M    W      H  NaN   
...            ...         ...   ...       ...   ...  ..  ...    ...  ...   
219216  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   
219217  12/31/2018  12/31/2018  2018  DECEMBER    53   F    W      H  NaN   
219218  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219219  12/31/2018  12/31/2018  2018  DECEMBER    53   M    B      N  NaN   
219222  12/31/2018  12/31/2018  2018  DECEMBER    53   M    W      H  NaN   

            SID  ... INVOLVEMENT  INCIDENT_NUM OFFENSE_CODE  \
0           

In [122]:
Crime_df=Crime_df.loc[(Crime_df["ZIP"]!="0") & (Crime_df["ZIP"]!="1") & (Crime_df["ZIP"]!="78") & 
                                (Crime_df["ZIP"]!="11111") & (Crime_df["ZIP"]!="6278") & (Crime_df["ZIP"]!="50701")
                               & (Crime_df["ZIP"]!="81507") & (Crime_df["ZIP"]!="70205") & (Crime_df["ZIP"]!="78200")]
print(Crime_df)

In [36]:
# of zipcodes
Zipcodes_df = Crime_df.loc[:,["ZIP"]]
print(Zipcodes_df)
print(type(Zipcodes_df))
Zipcodes_df["ZIP"].unique()

          ZIP
0       78205
1       78201
2       78218
3       78201
4       78207
...       ...
219219  78220
219220  78220
219221  78222
219222  78257
219223  78228

[217435 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


array(['78205', '78201', '78218', '78207', '78225', '78210', '78230',
       '78212', '78211', '78228', '78242', '78221', '78238', '78213',
       '78202', '78245', '78240', '78233', '78219', '78224', '78251',
       '78216', '78237', '78217', '78209', '78227', '78214', '78254',
       '78223', '78252', '78220', '78266', '78247', '78232', '78249',
       '78204', '78256', '78250', '78229', '78235', '78203', '78215',
       '78258', '78253', '78226', '78244', '78239', '78023', '78154',
       '78208', '78222', '78109', '78259', '78257', '78130', '78255',
       '78248', '78121', '78260', '78231', '78207-3170', '78236', '78241',
       '78155', '78070', '79502', '78233-6610', '78073', '78147', '78112',
       '78234', '78223-2099', '78016', '78000', '78264', '78148', '78263',
       '78056', '78006', '78205-2687', '78261', '78160', '78291', '78404',
       '78132', '78201-3847', '78108', '78026', '78009', '78238-4205',
       '78288', '78216-6902', '78054', '78163', '78015', '78002', '78

In [37]:
Num_of_Crimes_grp = Crime_df.groupby("ZIP")
Num_of_Crimes = Num_of_Crimes_grp["INCIDENT_NUM"].count()
Num_of_Crimes_2 = Num_of_Crimes.astype(float)
Num_of_Crimes_3 = pd.DataFrame(Num_of_Crimes_2)
print(type(Num_of_Crimes_3))
print(Num_of_Crimes_2)
print(Num_of_Crimes_3)

<class 'pandas.core.frame.DataFrame'>
ZIP
74134    1.0
78000    1.0
78002    3.0
78006    4.0
78009    1.0
        ... 
78288    8.0
78291    1.0
78404    1.0
78648    1.0
79502    3.0
Name: INCIDENT_NUM, Length: 104, dtype: float64
       INCIDENT_NUM
ZIP                
74134           1.0
78000           1.0
78002           3.0
78006           4.0
78009           1.0
...             ...
78288           8.0
78291           1.0
78404           1.0
78648           1.0
79502           3.0

[104 rows x 1 columns]


In [38]:
Zipcodes_df2 = pd.DataFrame(Zipcodes_df["ZIP"].unique(), columns=["Zip"])

Zipcodes_df2["Lat"] = ""
Zipcodes_df2["Lon"] = ""

# Target zipcode
# Zipcodes_df_2["Zip"] = Zipcodes_df_2["ZIP"].astype(str)

print(Zipcodes_df2.dtypes)
print(Zipcodes_df2)

Zip    object
Lat    object
Lon    object
dtype: object
       Zip Lat Lon
0    78205        
1    78201        
2    78218        
3    78207        
4    78225        
..     ...  ..  ..
99   78284        
100  78014        
101  78101        
102  74134        
103  78069        

[104 rows x 3 columns]


In [29]:

for index,row in Zipcodes_df2.iterrows():

    # get zipcode info from df
    Zipcode = row['Zip']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Zipcode}.")
    # Build the endpoint URL
    target_url = ("https://maps.googleapis.com/maps/api/place/textsearch/json?query="+str(Zipcode)+"&key="+g_key)
    response = requests.get(target_url).json()
    pprint.pprint(response)
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"The lat is: {results[0]['geometry']['location']['lat']}")
        print(f"The lng is: {results[0]['geometry']['location']['lng']}")
        
        Zipcodes_df2.loc[index, "Lat"] = response['results'][0]['geometry']['location']['lat']
        Zipcodes_df2.loc[index, "Lon"] = response['results'][0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Zipcodes_df2.to_csv(data)

Retrieving Results for Index 0: 78205.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78205, USA',
              'geometry': {'location': {'lat': 29.422658,
                                        'lng': -98.48699049999999},
                           'viewport': {'northeast': {'lat': 29.436588,
                                                      'lng': -98.468075},
                                        'southwest': {'lat': 29.4128519,
                                                      'lng': -98.497957}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78205',
              'place_id': 'ChIJGfaPt6pYXIYRjGJaIKQdkeQ',
              'reference': 'ChIJGfaPt6pYXIYRjGJaIKQdkeQ',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.422658
The lng is: -98.48699049999999
------------
Retrieving Results for Index 1: 78201.
{'html_attributions': [

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78228, USA',
              'geometry': {'location': {'lat': 29.455339,
                                        'lng': -98.56111419999999},
                           'viewport': {'northeast': {'lat': 29.489651,
                                                      'lng': -98.53743399999999},
                                        'southwest': {'lat': 29.4346031,
                                                      'lng': -98.6083069}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78228',
              'place_id': 'ChIJrSv2ACpcXIYRjVqbRej9iwA',
              'reference': 'ChIJrSv2ACpcXIYRjVqbRej9iwA',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.455339
The lng is: -98.56111419999999
------------
Retrieving Results for Index 10: 78242.
{'html_attributions': [],
 'results': [{'formatted_a

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78219, USA',
              'geometry': {'location': {'lat': 29.4489141,
                                        'lng': -98.39101269999999},
                           'viewport': {'northeast': {'lat': 29.551213,
                                                      'lng': -98.2644479},
                                        'southwest': {'lat': 29.419638,
                                                      'lng': -98.498592}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78219',
              'place_id': 'ChIJyUKncwHzXIYRqndMh4AjeQc',
              'reference': 'ChIJyUKncwHzXIYRqndMh4AjeQc',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.4489141
The lng is: -98.39101269999999
------------
Retrieving Results for Index 19: 78224.
{'html_attributions': [],
 'results': [{'formatted_address'

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78254, USA',
              'geometry': {'location': {'lat': 29.5330493,
                                        'lng': -98.78116449999999},
                           'viewport': {'northeast': {'lat': 29.6053239,
                                                      'lng': -98.6394919},
                                        'southwest': {'lat': 29.4954578,
                                                      'lng': -98.83414599999999}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78254',
              'place_id': 'ChIJqekXO4VpXIYRh60iGgkFg8Y',
              'reference': 'ChIJqekXO4VpXIYRh60iGgkFg8Y',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.5330493
The lng is: -98.78116449999999
------------
Retrieving Results for Index 28: 78223.
{'html_attributions': [],
 'results': [{'formatte

{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78256, USA',
              'geometry': {'location': {'lat': 29.6183126, 'lng': -98.6215649},
                           'viewport': {'northeast': {'lat': 29.6641971,
                                                      'lng': -98.60013909999999},
                                        'southwest': {'lat': 29.584583,
                                                      'lng': -98.6587529}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'San Antonio, TX 78256',
              'place_id': 'ChIJvadoPchlXIYRkV-BAKID6Vg',
              'reference': 'ChIJvadoPchlXIYRkV-BAKID6Vg',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.6183126
The lng is: -98.6215649
------------
Retrieving Results for Index 37: 78250.
{'html_attributions': [],
 'results': [{'formatted_address': 'San Antonio, TX 78250, USA',
             

...

{'html_attributions': [],
 'results': [{'formatted_address': 'Aspermont, TX 79502, USA',
              'geometry': {'location': {'lat': 33.152316, 'lng': -100.2151578},
                           'viewport': {'northeast': {'lat': 33.39805200000001,
                                                      'lng': -99.989301},
                                        'southwest': {'lat': 32.959801,
                                                      'lng': -100.517602}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Aspermont, TX 79502',
              'place_id': 'ChIJqXPnmu2XVYYRjKp2uHS5QJQ',
              'reference': 'ChIJqXPnmu2XVYYRjKp2uHS5QJQ',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 33.152316
The lng is: -100.2151578
------------
Retrieving Results for Index 66: 78233-6610.
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
---

{'html_attributions': [],
 'results': [{'formatted_address': 'Mico, TX 78056, USA',
              'geometry': {'location': {'lat': 29.5074919,
                                        'lng': -98.99079379999999},
                           'viewport': {'northeast': {'lat': 29.6237531,
                                                      'lng': -98.80606399999999},
                                        'southwest': {'lat': 29.4328409,
                                                      'lng': -99.104124}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Mico, TX 78056',
              'place_id': 'ChIJ92FICBQUXIYRuhn1jOR6ULw',
              'reference': 'ChIJ92FICBQUXIYRuhn1jOR6ULw',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.5074919
The lng is: -98.99079379999999
------------
Retrieving Results for Index 78: 78006.
{'html_attributions': [],
 'results': [{'formatted_address': 'FA

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
Missing field/result... skipping.
------------
Retrieving Results for Index 86: 78108.
{'html_attributions': [],
 'results': [{'formatted_address': 'Schertz, TX 78108, USA',
              'geometry': {'location': {'lat': 29.5682752, 'lng': -98.2322355},
                           'viewport': {'northeast': {'lat': 29.6369589,
                                                      'lng': -98.17075159999999},
                                        'southwest': {'lat': 29.5014192,
                                                      'lng': -98.2766473}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Schertz, TX 78108',
              'place_id': 'ChIJTaLFo4SUXIYRfjbruDdQreE',
              'reference': 'ChIJTaLFo4SUXIYRfjbruDdQreE',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.5682752
The lng is: -98.2322355
----------

{'html_attributions': [],
 'results': [{'formatted_address': 'Macdona, TX 78054, USA',
              'geometry': {'location': {'lat': 29.3261508,
                                        'lng': -98.69341250000001},
                           'viewport': {'northeast': {'lat': 29.3274671,
                                                      'lng': -98.6922929},
                                        'southwest': {'lat': 29.3245481,
                                                      'lng': -98.695516}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Macdona, TX 78054',
              'place_id': 'ChIJQRVafntFXIYRGMPQLgpkgoA',
              'reference': 'ChIJQRVafntFXIYRGMPQLgpkgoA',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 29.3261508
The lng is: -98.69341250000001
------------
Retrieving Results for Index 93: 78163.
{'html_attributions': [],
 'results': [{'formatted_address': 'Bul

{'html_attributions': [],
 'results': [{'formatted_address': 'Tulsa, OK 74134, USA',
              'geometry': {'location': {'lat': 36.1129513, 'lng': -95.7826597},
                           'viewport': {'northeast': {'lat': 36.16006,
                                                      'lng': -95.761641},
                                        'southwest': {'lat': 36.075569,
                                                      'lng': -95.86178}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
              'name': 'Tulsa, OK 74134',
              'place_id': 'ChIJ4fSqgUCLtocRUmFkoBHmpwE',
              'reference': 'ChIJ4fSqgUCLtocRUmFkoBHmpwE',
              'types': ['postal_code']}],
 'status': 'OK'}
The lat is: 36.1129513
The lng is: -95.7826597
------------
Retrieving Results for Index 105: 78069.
{'html_attributions': [],
 'results': [{'formatted_address': 'Somerset, TX 78069, USA',
              'geometry': {'location': 

In [30]:
Zipcodes_df2["Lat"] = pd.to_numeric(Zipcodes_df2["Lat"])
Zipcodes_df2["Lon"] = pd.to_numeric(Zipcodes_df2["Lon"])


locations = Zipcodes_df2[["Lat", "Lon"]]
Crime = Num_of_Crimes_3
print(len(locations))
print(len(Crime))

106
106


In [12]:
fig= gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Crime, dissipating=False,
                                point_radius=5)

fig.add_layer(heat_layer)

fig

InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.